This is the using the new 505-60%-70%-80%-90% synthetic file

Trial of XGBoost:

In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import joblib
import re
import os
import time


# Install XGBoost in Colab environment (run once per session)
print("Installing xgboost library...")
!pip install xgboost
print("XGBoost installed.")

# Import XGBoost
from xgboost import XGBClassifier


Installing xgboost library...
XGBoost installed.


(2) define the path of files:

In [ ]:

# --- Colab Specific Setup ---
# Mount Google Drive
print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# Define the base path to your project folder in Google Drive
# IMPORTANT: Make sure this path exactly matches where you uploaded your files
GOOGLE_DRIVE_PROJECT_PATH = '/content/drive/My Drive/2025-CS221 project'
print(f"Using Google Drive project path: {GOOGLE_DRIVE_PROJECT_PATH}")

# --- NLTK Data Check ---
print("\nChecking for NLTK data (wordnet, omw-1.4)...")
try:
    WordNetLemmatizer().lemmatize('running')
    print("NLTK data already present.")
except LookupError:
    print("NLTK data not found, downloading...")
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    print("NLTK data download complete.")

# --- Model Training and Saving ---
# Adjust MODEL_DIR to be within your Google Drive project path for persistence
MODEL_DIR = os.path.join(GOOGLE_DRIVE_PROJECT_PATH, 'model_XGBoost')
os.makedirs(MODEL_DIR, exist_ok=True)
print(f"Model artifacts will be stored in: {MODEL_DIR}")

MODEL_PATH = os.path.join(MODEL_DIR, 'best_xgb_model.joblib')
VECTORIZER_PATH = os.path.join(MODEL_DIR, 'tfidf_vectorizer.joblib')
LABEL_ENCODER_PATH = os.path.join(MODEL_DIR, 'label_encoder.joblib')
TRAINING_DATA_PATH = os.path.join(MODEL_DIR, 'training_data.joblib')


Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted.
Using Google Drive project path: /content/drive/My Drive/2025-CS221 project

Checking for NLTK data (wordnet, omw-1.4)...
NLTK data not found, downloading...


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


NLTK data download complete.
Model artifacts will be stored in: /content/drive/My Drive/2025-CS221 project/model_XGBoost


(3) model traininging -initialization

In [ ]:
# Initialize model and transformers to None
best_xgb = None
vectorizer = None
label_encoder = None
X_train = np.array([])
y_train = np.array([])
X_test = np.array([])
y_test = np.array([])

start_time = time.time()

# Attempt to load model artifacts first
print("\nAttempting to load pre-trained model and transformers...")
if os.path.exists(MODEL_PATH) and os.path.exists(VECTORIZER_PATH) and \
        os.path.exists(LABEL_ENCODER_PATH):
    try:
        best_xgb = joblib.load(MODEL_PATH)
        vectorizer = joblib.load(VECTORIZER_PATH)
        label_encoder = joblib.load(LABEL_ENCODER_PATH)
        if os.path.exists(TRAINING_DATA_PATH):
            X_train_loaded, y_train_loaded = joblib.load(TRAINING_DATA_PATH)
            X_train, y_train = X_train_loaded, y_train_loaded
        else:
            print("Warning: Training data (X_train, y_train) not found. This might affect MRR calculation or predict_top_diagnoses if it relies on training data structure.")
        print("Model and transformers loaded successfully.")
    except Exception as e:
        print(f"Error loading model artifacts: {e}. Proceeding with new model training.")
else:
    print("No pre-trained model found or incomplete artifacts.")




Attempting to load pre-trained model and transformers...
No pre-trained model found or incomplete artifacts.


(4) model training

In [ ]:
if best_xgb is None:
    print("Initiating new model training process...")
    print("Loading JSON data for training...")
    json_file_path = os.path.join(GOOGLE_DRIVE_PROJECT_PATH, 'synthetic_synopsis.json')
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    print("JSON data loaded.")

    symptoms = []
    diagnoses = []

    if isinstance(data, dict):
        lemmatizer = WordNetLemmatizer()
        custom_stop_words = set() # This remains empty, so no custom stop words are removed by it

        def preprocess_text(text):
            text = text.lower()
            tokens = text.split()
            tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stop_words]
            return " ".join(tokens)

        print("Processing JSON data to extract symptoms and diagnoses...")
        for syndrome, details in data.items():
            if isinstance(details, dict):
                diagnosis = syndrome
                for synopsis in details.get('synopsis', []):
                    if isinstance(synopsis, list):
                        clinical_synopsis = " ".join(synopsis).lower()
                        processed_synopsis = preprocess_text(clinical_synopsis)
                        symptoms.append(processed_synopsis)
                        diagnoses.append(diagnosis)
        print(f"Extracted {len(symptoms)} symptom-diagnosis pairs.")
    else:
        print("The data is not in the expected format. Please check 'synthetic_synopsis.json'. Exiting.")
        exit()

    df = pd.DataFrame({'symptoms': symptoms, 'diagnosis': diagnoses})
    print(f"Created DataFrame with {len(df)} entries.")

    X = df['symptoms']
    y = df['diagnosis']

    print("Encoding labels...")
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    print(f"Labels encoded for {len(label_encoder.classes_)} unique diagnoses.")

    print("Fitting TF-IDF vectorizer...")
    # --- FIX: Changed min_df from 5 to 1 ---
    vectorizer = TfidfVectorizer(max_features=5000, stop_words=list(custom_stop_words), ngram_range=(1, 2), min_df=1)
    X_tfidf = vectorizer.fit_transform(X).toarray()
    print(f"TF-IDF vectorizer fitted. X_tfidf shape: {X_tfidf.shape}")

    print("Splitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42,
                                                        stratify=y_encoded)
    print(f'Number of training samples: {X_train.shape[0]}')
    print(f'Number of testing samples: {X_test.shape[0]}')

    # --- XGBoost Hyperparameter Tuning ---
    param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.05, 0.1],
        'max_depth': [3, 5],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
        'gamma': [0.1],
        'eval_metric': ['mlogloss'],
        'device': ['cuda']
    }
    print(f"\nStarting GridSearchCV for XGBoost. This may take a while...")
    print(f"Parameters to tune: {param_grid}")
    skf = StratifiedKFold(n_splits=3)
    grid_search = GridSearchCV(XGBClassifier(objective='multi:softproba', random_state=42),
                               param_grid, cv=skf, scoring='neg_log_loss', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    best_xgb = grid_search.best_estimator_
    print(f'\nGridSearchCV complete. Best XGBoost parameters: {grid_search.best_params_}')

    print("Saving trained model and transformers...")
    joblib.dump(best_xgb, MODEL_PATH)
    joblib.dump(vectorizer, VECTORIZER_PATH)
    joblib.dump(label_encoder, LABEL_ENCODER_PATH)
    joblib.dump((X_train, y_train), TRAINING_DATA_PATH)
    print("Model, vectorizer, and label encoder saved successfully.")
else: # If model was loaded, ensure X_test and y_test are also defined from the original split logic
    print("Model loaded, preparing test set for evaluation...")
    json_file_path = os.path.join(GOOGLE_DRIVE_PROJECT_PATH, 'synthetic_synopsis.json')
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    symptoms_all = []
    diagnoses_all = []
    if isinstance(data, dict):
        lemmatizer = WordNetLemmatizer()
        custom_stop_words = set()
        def preprocess_text(text):
            text = text.lower()
            tokens = text.split()
            tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stop_words]
            return " ".join(tokens)
        for syndrome, details in data.items():
            if isinstance(details, dict):
                diagnosis = syndrome
                for synopsis in details.get('synopsis', []):
                    if isinstance(synopsis, list):
                        clinical_synopsis = " ".join(synopsis).lower()
                        processed_synopsis = preprocess_text(clinical_synopsis)
                        symptoms_all.append(processed_synopsis)
                        diagnoses_all.append(diagnosis)
    df_all = pd.DataFrame({'symptoms': symptoms_all, 'diagnosis': diagnoses_all})
    X_all = df_all['symptoms']
    y_all = df_all['diagnosis']
    y_all_encoded = label_encoder.transform(y_all)
    X_all_tfidf = vectorizer.transform(X_all).toarray()
    _, X_test, _, y_test = train_test_split(X_all_tfidf, y_all_encoded, test_size=0.2, random_state=42, stratify=y_all_encoded)
    print("Test set prepared for evaluation from loaded data.")


# --- Model Evaluation (always run after training or loading) ---
print("\n--- Evaluating the model on the test set ---")
print("Making predictions on the test set...")
y_pred = best_xgb.predict(X_test)
print("Predictions made.")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=0))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


def mean_reciprocal_rank(y_true, y_pred_proba, label_encoder, top_n=20):
    mrr = 0.0
    print(f"Calculating Mean Reciprocal Rank (MRR) for top {top_n}...")
    for i, true_label_encoded in enumerate(y_true):
        probabilities = y_pred_proba[i]
        sorted_indices_encoded = np.argsort(probabilities)[::-1]
        top_n_predicted_labels_encoded = sorted_indices_encoded[:top_n]

        true_original_label = label_encoder.inverse_transform([true_label_encoded])[0]
        top_n_predicted_labels_original = label_encoder.inverse_transform(top_n_predicted_labels_encoded)

        if true_original_label in top_n_predicted_labels_original:
            rank = np.where(top_n_predicted_labels_original == true_original_label)[0][0] + 1
            mrr += 1.0 / rank
    print("MRR calculation complete.")
    return mrr / len(y_true)

y_pred_proba = best_xgb.predict_proba(X_test)
mrr = mean_reciprocal_rank(y_test, y_pred_proba, label_encoder, top_n=20)
print(f'Mean Reciprocal Rank (Top 20): {mrr:.4f}')


def predict_top_diagnoses(symptom, top_n):
    lemmatizer = WordNetLemmatizer()
    custom_stop_words = set()

    def preprocess_text_for_prediction(text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        tokens = text.split()
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stop_words]
        return " ".join(tokens)

    processed_symptom = preprocess_text_for_prediction(symptom)

    if not processed_symptom.strip():
        return []

    if vectorizer is None or best_xgb is None or label_encoder is None:
        print("Error: Model, vectorizer, or label encoder not loaded/trained before prediction attempt.")
        return []

    symptom_tfidf = vectorizer.transform([processed_symptom]).toarray()
    probabilities = best_xgb.predict_proba(symptom_tfidf)[0]
    sorted_indices = np.argsort(probabilities)[::-1]
    top_n_predicted_encoded = sorted_indices[:top_n]
    top_n_diagnoses = label_encoder.inverse_transform(top_n_predicted_encoded)

    return top_n_diagnoses.tolist()


def process_and_evaluate_csv(file_path):
    full_file_path = os.path.join(GOOGLE_DRIVE_PROJECT_PATH, file_path)
    print(f"\n--- Starting evaluation for '{full_file_path}' ---")
    try:
        df = pd.read_csv(full_file_path)
        print(f"Successfully loaded '{full_file_path}'. Initial rows: {len(df)}")

        df['symptoms'] = df['symptoms'].astype(str).fillna('')
        df['diagnosis'] = df['diagnosis'].astype(str).fillna('')

        initial_rows = len(df)
        df = df[~((df['symptoms'] == '') & (df['diagnosis'] == ''))]
        rows_deleted = initial_rows - len(df)
        if rows_deleted > 0:
            print(f"Deleted {rows_deleted} rows from '{full_file_path}' where both 'symptoms' and 'diagnosis' were empty/NaN.")

        if df.empty:
            print(f"After cleaning, '{full_file_path}' has no valid cases to process. Skipping.")
            return

    except FileNotFoundError:
        print(f"Error: '{full_file_path}' not found. Please ensure the file is in the correct Google Drive path. Skipping.")
        return

    results = []
    top_n_output_20 = 20
    top_n_output_10 = 10

    print(f"Processing {len(df)} cases from '{full_file_path}'...")
    progress_interval = max(1, len(df) // 10)

    for index, row in df.iterrows():
        if (index + 1) % progress_interval == 0 or (index + 1) == len(df):
            print(f"  Processed {index + 1}/{len(df)} cases from '{full_file_path}'")

        symptoms_case = row['symptoms']
        true_diagnosis = str(row['diagnosis'])

        predicted_top_20_diagnoses = predict_top_diagnoses(symptoms_case, top_n=top_n_output_20)
        predicted_top_10_diagnoses = predict_top_diagnoses(symptoms_case, top_n=top_n_output_10)

        normalized_true_diagnosis = re.sub(r'\s+', ' ', true_diagnosis.strip().upper())

        normalized_predicted_20_diagnoses = [re.sub(r'\s+', ' ', d.strip().upper()) for d in predicted_top_20_diagnoses]
        normalized_predicted_10_diagnoses = [re.sub(r'\s+', ' ', d.strip().upper()) for d in predicted_top_10_diagnoses]

        match_20 = 1 if normalized_true_diagnosis in normalized_predicted_20_diagnoses else 0
        match_10 = 1 if normalized_true_diagnosis in normalized_predicted_10_diagnoses else 0

        results.append({
            'symptoms': symptoms_case,
            'diagnosis': true_diagnosis,
            'possible diagnosis (Top 20)': "; ".join(predicted_top_20_diagnoses),
            'match_top_20': match_20,
            'match_top_10': match_10
        })

    output_df = pd.DataFrame(results)

    output_csv_filename = os.path.join(GOOGLE_DRIVE_PROJECT_PATH, file_path.replace('.csv', '_predictions_xgb.csv'))
    output_df.to_csv(output_csv_filename, index=False)
    print(f"\nPredictions saved to '{output_csv_filename}'")

    total_cases = len(output_df)

    successful_matches_20 = output_df['match_top_20'].sum()
    unsuccessful_matches_20 = total_cases - successful_matches_20
    if total_cases > 0:
        prediction_rate_20 = successful_matches_20 / total_cases
        print(
            f"\nPrediction Rate for '{file_path}' (True diagnosis within top {top_n_output_20} possible diagnoses): {successful_matches_20}/({successful_matches_20}+{unsuccessful_matches_20}) = {prediction_rate_20:.2%}")
    else:
        print(f"\nNo valid cases processed from '{file_path}' to calculate prediction rate (Top {top_n_output_20}).")

    successful_matches_10 = output_df['match_top_10'].sum()
    unsuccessful_matches_10 = total_cases - successful_matches_10
    if total_cases > 0:
        prediction_rate_10 = successful_matches_10 / total_cases
        print(
            f"Prediction Rate for '{file_path}' (True diagnosis within top {top_n_output_10} possible diagnoses): {successful_matches_10}/({successful_matches_10}+{unsuccessful_matches_10}) = {prediction_rate_10:.2%}")
    else:
        print(f"\nNo valid cases processed from '{file_path}' to calculate prediction rate (Top {top_n_output_10}).")

    print(f"--- Finished evaluation for '{file_path}' ---")


# --- Main execution block ---
if __name__ == "__main__":
    process_and_evaluate_csv("clinical_cases_genetic_epilepsy.csv")
    process_and_evaluate_csv("pubmed_case_reports.csv")

    end_time = time.time()
    print(f"\nTotal script execution time: {end_time - start_time:.2f} seconds")

Initiating new model training process...
Loading JSON data for training...
JSON data loaded.
Processing JSON data to extract symptoms and diagnoses...
Extracted 13784 symptom-diagnosis pairs.
Created DataFrame with 13784 entries.
Encoding labels...
Labels encoded for 1723 unique diagnoses.
Fitting TF-IDF vectorizer...
TF-IDF vectorizer fitted. X_tfidf shape: (13784, 5000)
Splitting data into training and testing sets...
Number of training samples: 11027
Number of testing samples: 2757

Starting GridSearchCV for XGBoost. This may take a while...
Parameters to tune: {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 5], 'subsample': [0.8], 'colsample_bytree': [0.8], 'gamma': [0.1], 'eval_metric': ['mlogloss'], 'device': ['cuda']}
Fitting 3 folds for each of 8 candidates, totalling 24 fits
